In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline


data_dir = '/Users/boyuliu/pyprojects/Joann/Joann-Thailand-Project/notebooks/datasets/new_dataset/'

# wv1 = pd.read_csv(
#         data_dir + 'regression_data_%s_USD_20210311.csv' % 'wv_cases1')
# wv2 = pd.read_csv(
#         data_dir + 'regression_data_%s_USD_20210311.csv' % 'wv_cases2')
# wv3 = pd.read_csv(
#         data_dir + 'regression_data_%s_USD_20210311.csv' % 'wv_cases3')
# wv4 = pd.read_csv(
#         data_dir + 'regression_data_%s_USD_20210311.csv' % 'wv_cases4')
# wv1.head()

## calculate shock

In [9]:
wv_data_file = 'wv_cases1'
df = wv1
df['demand_shock'] = None

for prov in df.province.unique():
    prov_row_idx = df[df.province==prov].index 
    demand_data_points = df.loc[prov_row_idx, 'total_demand'].values
    std_demand = np.std(demand_data_points)
    df.loc[[0, 1], 'demand_shock'] = np.nan # need at least two points to fit a line
    
#     i = 0
    for time_pointer in range(2, len(prov_row_idx)):
        y = demand_data_points[max(0, time_pointer-4):time_pointer]
        expected_demand = np.mean(y)
        demand_shock = demand_data_points[time_pointer] - expected_demand
#         print(x, y, expected_demand, demand_shock)
        df.loc[prov_row_idx[time_pointer], 'demand_shock'] = demand_shock/std_demand

#         i+=1
#         if i>3: break

    
print(df.shape)


(6554, 43)


In [10]:

    # create placeholder
for offset in range(1, 9):
    df['demand_shock_plus_%s' % offset] = None
# shift IV the other way by up to 2 months    
for offset in range(1, 9):
    df['demand_shock_minus_%s' % offset] = None

for prov in df.province.unique():
    prov_row_idx = df[df.province==prov].index 

    for offset in range(1, 9):
        df.loc[prov_row_idx, 'demand_shock_plus_%s' % offset] = df.loc[prov_row_idx, 'demand_shock'].shift(offset)
    # shift IV the other way by up to 2 months    
    for offset in range(1, 9):
        df.loc[prov_row_idx, 'demand_shock_minus_%s' % offset] = df.loc[prov_row_idx, 'demand_shock'].shift(-offset)

print(df.shape)

(6554, 59)


In [11]:
df.head()

,province,year_week,total_demand,perc_abuse,wv_count,ex_rate,fake_date,month,quarter,ex_rate_diff,...,demand_shock_plus_7,demand_shock_plus_8,demand_shock_minus_1,demand_shock_minus_2,demand_shock_minus_3,demand_shock_minus_4,demand_shock_minus_5,demand_shock_minus_6,demand_shock_minus_7,demand_shock_minus_8
0,Ang Thong,2018-02,0.0,0.0,0.0,32.29036,2018-01-08,2018-01,2018-1,-0.188473,...,NaN,NaN,NaN,0,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763
1,Ang Thong,2018-03,0.0,0.0,0.0,32.08888,2018-01-15,2018-01,2018-1,-0.201480,...,NaN,NaN,0,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763
2,Ang Thong,2018-04,0.0,0.0,0.0,31.81748,2018-01-22,2018-01,2018-1,-0.271400,...,NaN,NaN,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763
3,Ang Thong,2018-05,10.0,0.0,0.0,31.53970,2018-01-29,2018-01,2018-1,-0.277780,...,NaN,NaN,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763,-0.368763
4,Ang Thong,2018-06,0.0,0.0,0.0,31.78962,2018-02-05,2018-02,2018-1,0.249920,...,NaN,NaN,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763,-0.368763,0


In [12]:
df.columns

Index(['province', 'year_week', 'total_demand', 'perc_abuse', 'wv_count',
       'ex_rate', 'fake_date', 'month', 'quarter', 'ex_rate_diff',
       'ex_rate_diff_plus_1', 'ex_rate_diff_plus_2', 'ex_rate_diff_plus_3',
       'ex_rate_diff_plus_4', 'ex_rate_diff_plus_5', 'ex_rate_diff_plus_6',
       'ex_rate_diff_plus_7', 'ex_rate_diff_plus_8', 'ex_rate_diff_minus_1',
       'ex_rate_diff_minus_2', 'ex_rate_diff_minus_3', 'ex_rate_diff_minus_4',
       'ex_rate_diff_minus_5', 'ex_rate_diff_minus_6', 'ex_rate_diff_minus_7',
       'ex_rate_diff_minus_8', 'total_demand_plus_1', 'total_demand_plus_2',
       'total_demand_plus_3', 'total_demand_plus_4', 'total_demand_plus_5',
       'total_demand_plus_6', 'total_demand_plus_7', 'total_demand_plus_8',
       'total_demand_minus_1', 'total_demand_minus_2', 'total_demand_minus_3',
       'total_demand_minus_4', 'total_demand_minus_5', 'total_demand_minus_6',
       'total_demand_minus_7', 'total_demand_minus_8', 'demand_shock',
       'demand

In [13]:
def yr_wk_to_float(yr_wk):
    yr, wk = yr_wk.split('-')
    return int(yr) + float(wk)/100

df['yr_wk_float'] = df.year_week.apply(yr_wk_to_float)

In [14]:
wv_data_file = 'wv_cases1'

df.to_csv(
    data_dir + 'regression_data_%s_causal_ma_detrend_USD_20210311.csv' % wv_data_file, 
    index=False)
df.head()

,province,year_week,total_demand,perc_abuse,wv_count,ex_rate,fake_date,month,quarter,ex_rate_diff,...,demand_shock_plus_8,demand_shock_minus_1,demand_shock_minus_2,demand_shock_minus_3,demand_shock_minus_4,demand_shock_minus_5,demand_shock_minus_6,demand_shock_minus_7,demand_shock_minus_8,yr_wk_float
0,Ang Thong,2018-02,0.0,0.0,0.0,32.29036,2018-01-08,2018-01,2018-1,-0.188473,...,NaN,NaN,0,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,2018.02
1,Ang Thong,2018-03,0.0,0.0,0.0,32.08888,2018-01-15,2018-01,2018-1,-0.201480,...,NaN,0,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,2018.03
2,Ang Thong,2018-04,0.0,0.0,0.0,31.81748,2018-01-22,2018-01,2018-1,-0.271400,...,NaN,0.983368,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763,2018.04
3,Ang Thong,2018-05,10.0,0.0,0.0,31.53970,2018-01-29,2018-01,2018-1,-0.277780,...,NaN,-0.245842,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763,-0.368763,2018.05
4,Ang Thong,2018-06,0.0,0.0,0.0,31.78962,2018-02-05,2018-02,2018-1,0.249920,...,NaN,-0.245842,-0.245842,1.22921,-0.368763,-0.368763,-0.368763,-0.368763,0,2018.06


In [15]:
'regression_data_%s_causal_ma_detrend_USD_20210311.csv' % wv_data_file

'regression_data_wv_cases1_causal_ma_detrend_USD_20210311.csv'

In [16]:
df.columns

Index(['province', 'year_week', 'total_demand', 'perc_abuse', 'wv_count',
       'ex_rate', 'fake_date', 'month', 'quarter', 'ex_rate_diff',
       'ex_rate_diff_plus_1', 'ex_rate_diff_plus_2', 'ex_rate_diff_plus_3',
       'ex_rate_diff_plus_4', 'ex_rate_diff_plus_5', 'ex_rate_diff_plus_6',
       'ex_rate_diff_plus_7', 'ex_rate_diff_plus_8', 'ex_rate_diff_minus_1',
       'ex_rate_diff_minus_2', 'ex_rate_diff_minus_3', 'ex_rate_diff_minus_4',
       'ex_rate_diff_minus_5', 'ex_rate_diff_minus_6', 'ex_rate_diff_minus_7',
       'ex_rate_diff_minus_8', 'total_demand_plus_1', 'total_demand_plus_2',
       'total_demand_plus_3', 'total_demand_plus_4', 'total_demand_plus_5',
       'total_demand_plus_6', 'total_demand_plus_7', 'total_demand_plus_8',
       'total_demand_minus_1', 'total_demand_minus_2', 'total_demand_minus_3',
       'total_demand_minus_4', 'total_demand_minus_5', 'total_demand_minus_6',
       'total_demand_minus_7', 'total_demand_minus_8', 'demand_shock',
       'demand

# extra iv labor abuse

In [3]:
df = pd.read_csv(
        data_dir + 'regression_data_wv_cases1_causal_ma_detrend_USD_20210311.csv')
print(df.shape)
df.head()

(6554, 60)


,province,year_week,total_demand,perc_abuse,wv_count,ex_rate,fake_date,month,quarter,ex_rate_diff,...,demand_shock_plus_8,demand_shock_minus_1,demand_shock_minus_2,demand_shock_minus_3,demand_shock_minus_4,demand_shock_minus_5,demand_shock_minus_6,demand_shock_minus_7,demand_shock_minus_8,yr_wk_float
0,Ang Thong,2018-02,0.0,0.0,0.0,32.29036,2018-01-08,2018-01,2018-1,-0.188473,...,NaN,NaN,0.000000,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,2018.02
1,Ang Thong,2018-03,0.0,0.0,0.0,32.08888,2018-01-15,2018-01,2018-1,-0.201480,...,NaN,0.000000,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,2018.03
2,Ang Thong,2018-04,0.0,0.0,0.0,31.81748,2018-01-22,2018-01,2018-1,-0.271400,...,NaN,0.983368,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,2018.04
3,Ang Thong,2018-05,10.0,0.0,0.0,31.53970,2018-01-29,2018-01,2018-1,-0.277780,...,NaN,-0.245842,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,-0.368763,2018.05
4,Ang Thong,2018-06,0.0,0.0,0.0,31.78962,2018-02-05,2018-02,2018-1,0.249920,...,NaN,-0.245842,-0.245842,1.229210,-0.368763,-0.368763,-0.368763,-0.368763,0.000000,2018.06


In [4]:
df['perc_abuse_minus_1'] = None 
df['perc_abuse_minus_2'] = None 
df['perc_abuse_minus_3'] = None 
df['perc_abuse_minus_4'] = None    
df['perc_abuse_minus_5'] = None   
df['perc_abuse_minus_6'] = None   
df['perc_abuse_minus_7'] = None   
for prov in df.province.unique():
    prov_row_idx = df[df.province==prov].index 
    df.loc[prov_row_idx, 'perc_abuse_minus_1'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-1)
    df.loc[prov_row_idx, 'perc_abuse_minus_2'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-2)
    df.loc[prov_row_idx, 'perc_abuse_minus_3'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-3)
    df.loc[prov_row_idx, 'perc_abuse_minus_4'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-4)
    df.loc[prov_row_idx, 'perc_abuse_minus_5'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-5)
    df.loc[prov_row_idx, 'perc_abuse_minus_6'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-6)
    df.loc[prov_row_idx, 'perc_abuse_minus_7'] = df.loc[prov_row_idx, 'perc_abuse'].shift(-7)
df.head()    

,province,year_week,total_demand,perc_abuse,wv_count,ex_rate,fake_date,month,quarter,ex_rate_diff,...,demand_shock_minus_7,demand_shock_minus_8,yr_wk_float,perc_abuse_minus_1,perc_abuse_minus_2,perc_abuse_minus_3,perc_abuse_minus_4,perc_abuse_minus_5,perc_abuse_minus_6,perc_abuse_minus_7
0,Ang Thong,2018-02,0.0,0.0,0.0,32.29036,2018-01-08,2018-01,2018-1,-0.188473,...,1.229210,-0.368763,2018.02,0,0,0,0,0,0,0
1,Ang Thong,2018-03,0.0,0.0,0.0,32.08888,2018-01-15,2018-01,2018-1,-0.201480,...,-0.368763,-0.368763,2018.03,0,0,0,0,0,0,0
2,Ang Thong,2018-04,0.0,0.0,0.0,31.81748,2018-01-22,2018-01,2018-1,-0.271400,...,-0.368763,-0.368763,2018.04,0,0,0,0,0,0,0
3,Ang Thong,2018-05,10.0,0.0,0.0,31.53970,2018-01-29,2018-01,2018-1,-0.277780,...,-0.368763,-0.368763,2018.05,0,0,0,0,0,0,0
4,Ang Thong,2018-06,0.0,0.0,0.0,31.78962,2018-02-05,2018-02,2018-1,0.249920,...,-0.368763,0.000000,2018.06,0,0,0,0,0,0,0


In [6]:
df.to_csv(
        data_dir + 'regression_data_wv_cases1_causal_ma_detrend_USD_20210322.csv', 
        index=False)

## sandbox

In [10]:
wv_data_file = 'wv_cases1'
df = wv1
df['demand_shock'] = None

for prov in df.province.unique():
    prov_row_idx = df[df.province==prov].index 
    demand_data_points = df.loc[prov_row_idx, 'total_demand'].values
    std_demand = np.std(demand_data_points)
    df.loc[[0, 1], 'demand_shock'] = np.nan # need at least two points to fit a line
    
    i = 0
    for time_pointer in range(2, len(prov_row_idx)):
        y = demand_data_points[max(0, time_pointer-4):time_pointer]
        expected_demand = np.mean(y)
        demand_shock = demand_data_points[time_pointer] - expected_demand
#         print(x, y, expected_demand, demand_shock)
        df.loc[prov_row_idx[time_pointer], 'demand_shock'] = demand_shock/std_demand
        import pdb; pdb.set_trace()
        i+=1
        if i>3: break

    
print(df.shape)


> <ipython-input-10-c68b0ace961a>(19)<module>()
-> i+=1
(Pdb) n
> <ipython-input-10-c68b0ace961a>(20)<module>()
-> if i>3: break
(Pdb) n
> <ipython-input-10-c68b0ace961a>(12)<module>()
-> for time_pointer in range(2, len(prov_row_idx)):
(Pdb) y
array([0., 0.])
(Pdb) expected_demand
0.0
(Pdb) demand_shock
0.0
(Pdb) n
> <ipython-input-10-c68b0ace961a>(13)<module>()
-> y = demand_data_points[max(0, time_pointer-4):time_pointer]
(Pdb) n
> <ipython-input-10-c68b0ace961a>(14)<module>()
-> expected_demand = np.mean(y)
(Pdb) n
> <ipython-input-10-c68b0ace961a>(15)<module>()
-> demand_shock = demand_data_points[time_pointer] - expected_demand
(Pdb) n
> <ipython-input-10-c68b0ace961a>(17)<module>()
-> df.loc[prov_row_idx[time_pointer], 'demand_shock'] = demand_shock/std_demand
(Pdb) n
> <ipython-input-10-c68b0ace961a>(18)<module>()
-> import pdb; pdb.set_trace()
(Pdb) demand_shock
10.0
(Pdb) expected_demand
0.0
(Pdb) n
> <ipython-input-10-c68b0ace961a>(19)<module>()
-> i+=1
(Pdb) n
> <ipython-in

BdbQuit: 